# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!Powershell.exe -Command type publications.tsv -Head 3

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2025-03-14	Risk assessment of essential oils on honey bees (Apis mellifera L.)	Insects	Open access!	"Caren, J., Y.-C. Zhu, Q. D. Read, and Y. Du. 2025. Risk assessment of essential oils on honey bees (Apis mellifera L.). Insects 16(3):303. DOI: 10.3390/insects16030303."	caren-et-al-2025	https://doi.org/10.3390/insects16030303
2025-03-01	Integrated effects of tillage, fertilizer sources and a rye cover crop on dryland cotton production	Agronomy Journal		"Adeli, A., J. P. Brooks, D. M. Miles, Q. D. Read, Y. Huang, G. Feng, and J. N. Jenkins. 2025. Integrated effects of tillage, fertilizer sources and a rye cover crop on dryland cotton production. Agronomy Journal 117(2):e70019. DOI: 10.1002/agj2.70019."	adeli-et-al-2025	https://doi.org/10.1002/agj2.70019


## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [6]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications.head()


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2025-03-14,Risk assessment of essential oils on honey bee...,Insects,Open access!,"Caren, J., Y.-C. Zhu, Q. D. Read, and Y. Du. 2...",caren-et-al-2025,https://doi.org/10.3390/insects16030303
1,2025-03-01,"Integrated effects of tillage, fertilizer sour...",Agronomy Journal,NaN,"Adeli, A., J. P. Brooks, D. M. Miles, Q. D. Re...",adeli-et-al-2025,https://doi.org/10.1002/agj2.70019
2,2024-12-21,Effect of fast and slow-acting bait toxicants ...,Journal of Economic Entomology,NaN,"Oi, D. H., R. A. Atchison, and Q. D. Read. 202...",oi-et-al-2024,https://doi.org/10.1093/jee/toae284
3,2025-01-08,Pyrrocidines A and B demonstrate synergistic i...,Frontiers in Microbiology,Open access!,"Lofton, L. W., Q. D. Read, H. L. Hamilton, A. ...",lofton-et-al-2025,https://doi.org/10.3389/fmicb.2024.1480920
4,2025-01-08,Immune responses of chickens against recombina...,Veterinary Immunology and Immunopathology,NaN,"Yeh, H.-Y. and Q. D. Read. 2025. Immune respon...",yeh-et-al-2025,https://doi.org/10.1016/j.vetimm.2024.110870


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [7]:
# Here, subset the rows of publications to only 1 or however many we need to update right now
publications = publications.head(2)

In [8]:
# Some test code to see what the filename would be
str(publications['pub_date'])

'0    2025-03-14\n1    2025-03-01\nName: pub_date, dtype: object'

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [9]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = str(item.pub_date)[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    ##md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [10]:
%%cmd
dir /s /o:-d ..\_publications\ 

Microsoft Windows [Version 10.0.22631.4890]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\Quentin.Read\Documents\GitHub\qdread.github.io\markdown_generator>dir /s /o:-d ..\_publications\ 
 Volume in drive C is OS
 Volume Serial Number is 5EC1-8538

 Directory of C:\Users\Quentin.Read\Documents\GitHub\qdread.github.io\_publications

03/14/2025  10:11 AM               616 2025-03-01-adeli-et-al-2025.md
03/14/2025  10:11 AM    <DIR>          .
03/14/2025  10:11 AM               543 2025-03-14-caren-et-al-2025.md
03/14/2025  09:50 AM    <DIR>          ..
03/14/2025  09:50 AM               729 2025-01-08-yeh-et-al-2025.md
03/14/2025  09:50 AM               687 2025-01-08-lofton-et-al-2025.md
03/14/2025  09:50 AM               667 2024-12-31-mears-et-al-2024.md
03/14/2025  09:50 AM               647 2024-12-30-frazier-et-al-2024.md
03/14/2025  09:50 AM               699 2024-09-18-pokoo-aikins-et-al-2024.md
03/14/2025  09:50 AM               643 2024-12-21-oi-et-al-2024.md